# <center> Finding the ground state energy of a two qubits system via VQE algorithm </center>

<div style="text-align: justify" >
As it is well-known, the VQE algorithm makes use of the variational principle to approximate the ground state of a given quantum hamiltonian, $H$ <a href = https://www.nature.com/articles/ncomms5213?origin=ppub>[1]</a>. To make a long story short, the variational principle ensures that, given a certain quantum state $|\psi \rangle$, it satisfies the following inequivalence (we assume normalized kets):
</div>

$$
\langle \psi | H |\psi \rangle \leq E_0 \ ,
$$

<div style="text-align: justify">
where we denoted with $E_0$ the ground state energy and the equivalence is satisfied if $|\psi \rangle$ actually corresponds to ground state of the system.

It is immediate to see that the variational principle provides an effective way to estimate the ground state of the system under investigation: by simply taking a <i>set</i> of kets $|\psi(\alpha , \, \beta , \, \dots) \rangle$, depending on some parameters $(\alpha , \, \beta , \, \dots)$, one can find the best approximation to the ground state by simply evaluating the expression $\langle \psi (\alpha , \, \beta , \, \dots) | H |\psi (\alpha , \, \beta , \, \dots) \rangle$ and looking for its minimum. Of course, in order to find a good approximation to the ground state, one has to choose properly the ansatz $|\psi(\alpha , \, \beta , \, \dots) \rangle$.

Our goal in this notebook, is to consider some simple $4 \times 4$ matrices and to give an estimate of their ground state energies, via a VQE algorithm written using Qiskit. The analysis performed in this notebook can be considered as a very small extension to the case of two qubits of the nice analysis of <a href = https://github.com/DavitKhach/quantum-algorithms-tutorials/blob/master/variational_quantum_eigensolver.ipynb>[2]</a>, to which we refer for a more detailed explanation of the basis of the VQE algorithm.
</div>

Let us start by constructing a first example of matrix to diagonalize. It reads

$$
   M=
  \left[ {\begin{array}{cccc}
   0 & 0 & 0 & 0 \\
   0 & -1 & 1 & 0 \\
   0 & 1 & -1 & 0 \\
   0 & 0 & 0 & 0 \\
  \end{array} } \right] \ ,
$$


<div style="text-align: justify">
and, to start with, we need to decompose it in terms of tensor products of Pauli operators.
Given that the matrix is $4 \times 4$, it means that can be written as sum of tensor products of two Pauli operators. In other words, it can be viewed as a two-qubits operator.
</div>

<div style="text-align: justify">
Here we put all the packages we need:
</div>

In [1]:
import numpy as np
from random import random
from scipy.optimize import minimize
from sympy.solvers import solve
from sympy import Symbol

from qiskit import *
from qiskit.aqua.algorithms import ExactEigensolver
from qiskit.quantum_info.operators import Operator , Pauli
from qiskit.aqua.operators import WeightedPauliOperator

<div style="text-align: justify">
Let us construct the matrix of the coefficients, as a numpy array:
</div>

In [2]:
M = np.array([[0. , 0. , 0. , 0.], [0. , -1. , 1. , 0.] , [0. , 1. , -1. , 0.] , [0. , 0. , 0. , 0.]])

<div style="text-align: justify">
We now define a function which transforms a $4 \times 4$ matrix to the tensor decomposition in terms of the Pauli operators.
</div>

In [3]:
def hamiltonian_operator(matrix):
    list_labels = []
    pauli_operator_list = []
    symbols_list = []
    equations = []
    list_dict = []
    unknown_matrix = 0
    #these are the building blocks of the Pauli objects we are going to have, like II
    pauli_names = ('I','X','Y','Z') 
    for i in range(0,4):
        for j in range(0,4):
            #we create the list of the labels
            list_labels.append(pauli_names[i] + pauli_names[j])
            pauli_operator_list.append(Operator(Pauli(label=pauli_names[i] + pauli_names[j])))
    for i in range(0,16):
        # we create the list of symbols, which we need to solve the equations below
        symbols_list.append(Symbol(list_labels[i]))   
    for i in range(0,16):
        # we create the matrix containing the various pauli operators and the unknown
        unknown_matrix = unknown_matrix + (Symbol(list_labels[i])) * pauli_operator_list[i].data 
    for i in range(0, 4):
        for j in range(0,4):
            # we create the list of equations
            equations.append(unknown_matrix[i,j] - matrix[i,j]) 
    sol = solve(equations , symbols_list, manual = True)
    # we create the dictionary to give to the WeightedPauliOperator 
    for i in range(0,16):
        list_dict.append({"coeff": {"imag": 0.0, "real": sol[0][i]}, "label": list_labels[i]})
    pauli_dict = {
        'paulis': list_dict
    } 
    return WeightedPauliOperator.from_dict(pauli_dict)

<div style="text-align: justify">
and we use this function just defined to transform our matrix $M$ in the sum of Pauli operators
</div>

In [4]:
H = hamiltonian_operator(M)

<div style="text-align: justify">
For future comparison, we compute the ground state energy via the ExactEigensolver class
</div>

In [5]:
exact_eigensolver = ExactEigensolver(H)
result_ee = exact_eigensolver.run()
reference_energy = result_ee['energy']

<div style="text-align: justify">
As already mentioned, a crucial ingredient in the VQE algorithms is the choice of the ansatz: indeed, it is important to choose an ansatz which includes the ground state or, at least, which can be very close to the true ground state of the system, in order to have a good estimate of the ground state energy.

Restricting our attention to the problem under consideration, it is immediate to recognize that the ground state is given by the state
</div>

$$
| \psi_0 \rangle = \frac{1}{\sqrt 2} \Bigl( | 10 \rangle - | 01 \rangle \Bigr) \ ,
$$

<div style="text-align: justify">
where, with the notation $| 10 \rangle$ we mean the states defined in the basis with definite spin along the $z$-axes.

It is immediate to see that to build the ground state via a quantum cirquit starting from the state $| 00 \rangle$, a Hadamard gate, and a subsequent CNOT gate are necessary, in order to make a superposition and to create the entanglement. Hence, we will work with the following ansatz:
</div>

$$
\Bigl( R_z (\alpha) \, I \Bigr)\Bigl( R_x(\beta) \, I \Bigr) CX \Bigl( HI \Bigr) \, | 00 \rangle \ ,
$$

<div style="text-align: justify">
where the last two gates determine the relative angle betweent the two qubits. Given this ansatz, we now define a function which, given two parameters, prepares the quantum state associated to the snsatz, together with the two classical bits for the measurements.
</div>

In [6]:
def quantum_ansatz(param):
    circ = QuantumCircuit(2,2)
    # Add a H gate on qubit 0, putting this qubit in a superposition.
    circ.h(0)
    # Add a CX (CNOT) gate on control qubit 0 and target qubit 1, putting
    # the qubits in a Bell state and creating then entanglement.
    circ.cx(0, 1)
    # and now we create the part of the Ansatz controlled by the parameters:
    #we do an Rx rotation and an Rz rotation (in this order!)
    circ.rx(param[0],0)
    circ.rz(param[1],0)
    return circ

<div style="text-align: justify">
Although the Qiskit built-in method .to_dict() allows us to convert our hamiltonian H (which is a WeightedPauliOperator) to a dictionary of this form 
</div>

In [7]:
dict_long = H.to_dict()
dict_long

{'paulis': [{'label': 'II', 'coeff': {'real': -0.5, 'imag': 0.0}},
  {'label': 'XX', 'coeff': {'real': 0.5, 'imag': 0.0}},
  {'label': 'YY', 'coeff': {'real': 0.5, 'imag': 0.0}},
  {'label': 'ZZ', 'coeff': {'real': 0.5, 'imag': 0.0}}]}

<div style="text-align: justify">
we prefer to prepare a new function to get from H a list of dictionaries, a bit lighter, called pauli_dict 
</div>

In [8]:
def pauli_operator_to_dict(pauli_operator):
    
    """
    This function eats our hamiltonian, constructed previously, and converts it to a list of dictionaries:
    [{'label': 'II', 'coeff': -0.5},
 {'label': 'XX', 'coeff': 0.5},
 {'label': 'YY', 'coeff': 0.5},
 {'label': 'ZZ', 'coeff': 0.5}].
    """
    
    dict_long = pauli_operator.to_dict()
    paulis = dict_long['paulis']
    paulis_dict = []

    for x in paulis:
        label = x['label']
        coeff = x['coeff']['real']
        paulis_dict.append({'label' : label , 'coeff' : coeff})

    return paulis_dict
pauli_dict = pauli_operator_to_dict(H)

<div style="text-align: justify">
When working with dictionaries, it is good to have a function which, given a label, returns the value associated to that label or returns $0$ if that label does not exist in the dictionary
</div>

In [9]:
def get_or_else_zero(d: dict, key: str):
    
    """
    Utility for working with dictionaries. If key is missing
    than return 0 otherwise the corresponding value.
    :param dict: the dictionary.
    :param key: key (string) in interest.
    :return: 0 or value of corresponding key.
    """
    
    value = 0
    if key in d:
        value = d[key]
    return value

<div style="text-align: justify">
Now, we define a function that, given a circuit, a qubit, and a measurement to do, appends the gate associated with the measurements we are going to do on the qubit and connect the qubit to the corresponding classical bit
</div>

In [10]:
def measurement_preparation_circuit(circuit, measure : str, qubit):
    
    """
    Create the preparation for a measurement: it adds the gates 
    necessary to convert the circuit to the appropriate 
    computational basis and connect the qubits with the corresponding classical bits
    """

    if measure == 'Z':
        circuit.measure(qubit, qubit)
    elif measure == 'X':
        circuit.u2(qubit, np.pi, qubit)
        circuit.measure(qubit, qubit)
    elif measure == 'Y':
        circuit.u2(qubit, np.pi/2, qubit)
        circuit.measure(qubit, qubit)
    
    return circuit

<div style="text-align: justify">
Let us now define the function associated with the quantum part of the algorithm. This function prepares the quantum circuit and performs the quantum experiment associated with a certain measurement. In the case the measurement involves an identity operator on one of the two qubits, we do the measurement on the other qubit only
</div>

In [11]:
def quantum_module(parameters, measure):
    
    # we check the presence of identity operators    
    if measure == 'II':
        return 1 # if both operators are identity there is nothing to measure
    elif measure[0] == 'I':
        check_identity = 2
    elif measure[1] == 'I':
        check_identity = 1
    else:
        check_identity = 0
    
    # quantum state preparation
    circuit = quantum_ansatz(parameters)
    
    for i in range(0,2):
        circuit = measurement_preparation_circuit(circuit, measure[i] , i)
    
    shots = 2000
    backend = BasicAer.get_backend('qasm_simulator')
    job = execute(circuit, backend, shots=shots)
    result = job.result()
    counts = result.get_counts()
    
    if check_identity == 0:
        expectation_value = (get_or_else_zero(counts, '00') + get_or_else_zero(counts, '11')  - get_or_else_zero(counts,'01') -  get_or_else_zero(counts,'10')) / shots
    elif check_identity == 2:
        expectation_value = (get_or_else_zero(counts, '00') - get_or_else_zero(counts,'10')) / shots
    elif check_identity == 1:
        expectation_value = (get_or_else_zero(counts, '00') - get_or_else_zero(counts,'01')) / shots
    
    return expectation_value

<div style="text-align: justify">
We now define the main function, which sums the results of the measurements perfomed on the various terms of the hamiltonian
</div>

In [12]:
def vqe(parameters):
    
    classical_adder = 0
        
    for i in range(0, len(pauli_dict)):
        classical_adder = classical_adder + pauli_dict[i]['coeff'] * quantum_module(parameters, pauli_dict[i]['label'])
    
    return classical_adder

<div style="text-align: justify">
and finally, we are ready to compute the ground state with the VQE algorithm. We use two methods: in the first we simply scan all the values of the parameters for our ansatz, in the second we use the minimize() method of scipy.
</div>

In [13]:
n_spacing_one = 40
n_spacing_two = 40
parameters_array_one = np.linspace(0, 2 * np.pi, n_spacing_one)
parameters_array_two = np.linspace(0, np.pi, n_spacing_two)
sol_list = []

for i in range(0 , n_spacing_one):
    for j in range(0 , n_spacing_two):
        sol_list.append(vqe([parameters_array_one[i] , parameters_array_one[j]]))

print('The exact ground state energy is: {}'.format(reference_energy))
print('The estimated ground state energy using the VQE algorithm is: {}'.format((sorted(sol_list))[0]))

The exact ground state energy is: -2.0
The estimated ground state energy using the VQE algorithm is: -1.9975


In [14]:
parameters_array = np.array([np.pi, np.pi]) # starting point for the parameters
tol = 1e-5 # tolerance for the required precision.

vqe_result = minimize(vqe, parameters_array, method = 'Powell',tol=tol)
print('The exact ground state energy is: {}'.format(reference_energy))
print('The estimated ground state energy from VQE algorithm and by minimization method is: {}'.format(vqe_result.fun))

The exact ground state energy is: -2.0
The estimated ground state energy from VQE algorithm and by minimization method is: -2.0


<div style="text-align: justify">
as we can see, the quality of the approximation is extremely good. This is not surprising, since the ground state falls in the ansatz we used.
</div>

<div style="text-align: justify">
We can generalize a bit our algorithm and consider more general matrices: we keep a matrix with the same structure of non-vanishing coefficients, but we replace the coefficients with random numbers (but we impose the restriction that the matrix is symmetric). We can also add few terms on the main diagonal
</div>

In [15]:
a = random()
b = random()

M = np.array([[a , 0. , 0 , 0.], [0. , a , b , 0.] , [0 , b , a , 0.] , [0. , 0. , 0. , b]])
H = hamiltonian_operator(M)

exact_eigensolver = ExactEigensolver(H)
result_ee = exact_eigensolver.run()
reference_energy = result_ee['energy']

pauli_dict = pauli_operator_to_dict(H)
pauli_dict

[{'label': 'II', 'coeff': 0.6761974355732028},
 {'label': 'IZ', 'coeff': 0.09643075333862625},
 {'label': 'XX', 'coeff': 0.193452587778662},
 {'label': 'YY', 'coeff': 0.193452587778662},
 {'label': 'ZI', 'coeff': 0.09643075333862625},
 {'label': 'ZZ', 'coeff': -0.09643075333862625}]

In [16]:
n_spacing_one = 40
n_spacing_two = 40
parameters_array_one = np.linspace(0, 2 * np.pi, n_spacing_one)
parameters_array_two = np.linspace(0, np.pi, n_spacing_two)
sol_list = []

for i in range(0 , n_spacing_one):
    for j in range(0 , n_spacing_two):
        sol_list.append(vqe([parameters_array_one[i] , parameters_array_one[j]]))

print('The exact ground state energy is: {}'.format(reference_energy))
print('The estimated ground state energy using the VQE algorithm is: {}'.format((sorted(sol_list))[0]))

The exact ground state energy is: 0.38572301335450504
The estimated ground state energy using the VQE algorithm is: 0.381385402697571


<div style="text-align: justify">
Once again, the approximation is pretty good in finding the ground state energy. However, notice that by taking a slightly more general matrix, our Ansatz for the ground state stops its validity and the approximation to the ground state is very bad.
</div>

In [17]:
a = random()
b = random()
c = random()

M = np.array([[0 , 0. , c , 0.], [0. , a , b , 0.] , [c , b , a , 0.] , [0. , 0. , 0. , 0]])
H = hamiltonian_operator(M)

exact_eigensolver = ExactEigensolver(H)
result_ee = exact_eigensolver.run()
reference_energy = result_ee['energy']

pauli_dict = pauli_operator_to_dict(H)
pauli_dict

[{'label': 'II', 'coeff': 0.374377988110401},
 {'label': 'XI', 'coeff': 0.3654025704062155},
 {'label': 'XX', 'coeff': 0.4733816162542325},
 {'label': 'XZ', 'coeff': 0.3654025704062155},
 {'label': 'YY', 'coeff': 0.4733816162542325},
 {'label': 'ZZ', 'coeff': -0.374377988110401}]

In [18]:
n_spacing_one = 40
n_spacing_two = 40
parameters_array_one = np.linspace(0, 2 * np.pi, n_spacing_one)
parameters_array_two = np.linspace(0, np.pi, n_spacing_two)
sol_list = []

for i in range(0 , n_spacing_one):
    for j in range(0 , n_spacing_two):
        sol_list.append(vqe([parameters_array_one[i] , parameters_array_one[j]]))

print('The exact ground state energy is: {}'.format(reference_energy))
print('The estimated ground state energy using the VQE algorithm is: {}'.format((sorted(sol_list))[0]))

The exact ground state energy is: -0.6738697832124032
The estimated ground state energy using the VQE algorithm is: -0.2599258073107657


[1] [A. Peruzzo et al., Nature Communications, "A variational eigenvalue solver on a photonic quantum processor" (2014).](https://www.nature.com/articles/ncomms5213?origin=ppub)

[2] [D. Khachatryan, "Variational quantum eigensolver".](https://github.com/DavitKhach/quantum-algorithms-tutorials/blob/master/variational_quantum_eigensolver.ipynb)